In [24]:
import pandas as pd
import numpy as np

In [25]:
df = pd.read_csv('stockdata (1).csv')
df.head()

,title,upvotes,num_comments,Date,Movement,sentence_embeddings
0,I predict Amazon will acquire Lyft in the next...,2973,388,2018-10-31,1,[-8.41741040e-02 -3.72491270e-01 1.18203712e+...
1,Elon Musk plans to bring out IPO for SpaceX's ...,1517,316,2020-02-06,0,[-0.18144539 0.05652054 0.47810453 0.30172 ...
2,JPMorgan announces big moves to support enviro...,1684,148,2020-02-25,0,[-1.05110221e-01 8.34366918e-01 3.17408562e-...
3,Trump is requesting a stimulus that would be t...,2350,856,2020-03-11,0,[-5.18182874e-01 1.88033581e-01 -1.24653853e-...
4,The US stock market has now wiped out the enti...,5300,727,2020-03-13,0,[-3.74976963e-01 1.03718734e+00 -9.58576560e-...


In [26]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [27]:
def to_lowercase(text):
    return text.lower()


def remove_urls(text):
    return re.sub(r'http\S+|www.\S+', '', text)


def remove_punctuation(text):
    return re.sub(r'[^\w\s]', '', text)

def remove_numbers(text):
    return re.sub(r'\d+', '', text)


def remove_special_characters(text):
    return re.sub(r'[^A-Za-z\s]', '', text)


stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word not in stop_words]
    return ' '.join(filtered_text)

from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

def lemmatize_words(text):
    word_tokens = word_tokenize(text)
    lemmatized_text = [lemmatizer.lemmatize(word) for word in word_tokens]
    return ' '.join(lemmatized_text)

def clean_text(text):
    text = to_lowercase(text)
    text = remove_urls(text)
    text = remove_punctuation(text)
    text = remove_numbers(text)
    text = remove_special_characters(text)
    text = remove_stopwords(text)
    text = lemmatize_words(text)
    return text

df['title'] = df['title'].apply(clean_text)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [28]:
!pip install vaderSentiment

In [29]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

def get_sentiment(sentences):
    sentiment_score = 0
    for sentence in sentences:
        sentiment = analyzer.polarity_scores(sentence)
        sentiment_score += sentiment['compound']
    return sentiment_score

df['sentiment'] = df['title'].apply(get_sentiment)

In [30]:
df

,title,upvotes,num_comments,Date,Movement,sentence_embeddings,sentiment
0,predict amazon acquire lyft next month,2973,388,2018-10-31,1,[-8.41741040e-02 -3.72491270e-01 1.18203712e+...,0.4588
1,elon musk plan bring ipo spacexs starlink proj...,1517,316,2020-02-06,0,[-0.18144539 0.05652054 0.47810453 0.30172 ...,1.8352
2,jpmorgan announces big move support environmen...,1684,148,2020-02-25,0,[-1.05110221e-01 8.34366918e-01 3.17408562e-...,1.3764
3,trump requesting stimulus would twice big obam...,2350,856,2020-03-11,0,[-5.18182874e-01 1.88033581e-01 -1.24653853e-...,0.9176
4,u stock market wiped entire trillion value gai...,5300,727,2020-03-13,0,[-3.74976963e-01 1.03718734e+00 -9.58576560e-...,1.8352
...,...,...,...,...,...,...,...
750,japan stock rebound historic loss asia market ...,1479,175,2024-08-06,1,[-7.06196487e-01 1.10559799e-02 -7.97872767e-...,0.9176
751,trump medium report million loss quarter reven...,5974,461,2024-08-09,1,[-6.02605164e-01 6.82640076e-01 -6.43867433e-...,2.2940
752,bloomberg u considers rare antitrust move brea...,3307,603,2024-08-13,1,[-4.11342978e-01 1.21175790e+00 2.05229864e-...,0.9176
753,starbucks giving incoming ceo niccol cash stoc...,3113,673,2024-08-15,1,[-1.05236697e+00 1.20822835e+00 -1.09947836e+...,1.3764


In [20]:
df['sentiment'] = (df['sentiment'] - df['sentiment'].min()) / (df['sentiment'].max() - df['sentiment'].min())

In [22]:
df['sentiment']

0      0.090909
1      0.363636
2      0.272727
3      0.181818
4      0.363636
         ...   
750    0.181818
751    0.454545
752    0.181818
753    0.272727
754    0.000000
Name: sentiment, Length: 755, dtype: float64

In [32]:
df['sentiment'] = (df['sentiment'] >= 0.5).astype(int)

In [34]:
df['sentiment']

0      0
1      1
2      1
3      1
4      1
      ..
750    1
751    1
752    1
753    1
754    0
Name: sentiment, Length: 755, dtype: int64

In [36]:
df

,title,upvotes,num_comments,Date,Movement,sentence_embeddings,sentiment
0,predict amazon acquire lyft next month,2973,388,2018-10-31,1,[-8.41741040e-02 -3.72491270e-01 1.18203712e+...,0
1,elon musk plan bring ipo spacexs starlink proj...,1517,316,2020-02-06,0,[-0.18144539 0.05652054 0.47810453 0.30172 ...,1
2,jpmorgan announces big move support environmen...,1684,148,2020-02-25,0,[-1.05110221e-01 8.34366918e-01 3.17408562e-...,1
3,trump requesting stimulus would twice big obam...,2350,856,2020-03-11,0,[-5.18182874e-01 1.88033581e-01 -1.24653853e-...,1
4,u stock market wiped entire trillion value gai...,5300,727,2020-03-13,0,[-3.74976963e-01 1.03718734e+00 -9.58576560e-...,1
...,...,...,...,...,...,...,...
750,japan stock rebound historic loss asia market ...,1479,175,2024-08-06,1,[-7.06196487e-01 1.10559799e-02 -7.97872767e-...,1
751,trump medium report million loss quarter reven...,5974,461,2024-08-09,1,[-6.02605164e-01 6.82640076e-01 -6.43867433e-...,1
752,bloomberg u considers rare antitrust move brea...,3307,603,2024-08-13,1,[-4.11342978e-01 1.21175790e+00 2.05229864e-...,1
753,starbucks giving incoming ceo niccol cash stoc...,3113,673,2024-08-15,1,[-1.05236697e+00 1.20822835e+00 -1.09947836e+...,1


In [37]:
#Accuracy
correct_predictions = (df['Movement'] == df['sentiment']).sum()
total_predictions = len(df)
accuracy = correct_predictions / total_predictions


In [38]:
accuracy

0.5456953642384106